In [28]:
spark.stop()

# Set paths

DEVEL_SRC must contain the directory use-cases and pykhaos

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when '-' is removed from name, remove also this line and adapt imports 
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
# AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
# if AMDOCS_SRC not in sys.path: 
#     sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
        
from project.project_generic import Project

import pykhaos.utils.notebooks as nb

import time
start_time = time.time()

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    EXTERNAL_LIB = os.path.join(os.environ.get('BDA_USER_HOME', ''), "lib", "external_libs")
    if EXTERNAL_LIB not in sys.path:
        sys.path.append(EXTERNAL_LIB)
    # feel free from commenting this line and the other ones that begin with "%%notify" if you do not have 
    # the extension installed or copy de lib from /var/SP/data/home/csanc109/lib/external_libs/jupyternotify/
    %load_ext jupyternotify 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

20190828-111810 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20190828_111810.log


<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, asc, to_date, create_map, sum as sql_sum
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
from churn.datapreparation.general.data_loader import get_unlabeled_car, get_port_requests_table, get_numclients_under_analysis
from churn.utils.constants import PORT_TABLE_NAME
from churn.utils.udf_manager import Funct_to_UDF
from pyspark.sql.functions import substring, datediff, row_number
from pykhaos.utils.date_functions import move_date_n_days, move_date_n_cycles
from pykhaos.utils.hdfs_functions import check_hdfs_exists
from pykhaos.modeling.model_performance import get_lift

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Functions

## Start spark context

In [4]:
from churn.utils.general_functions import init_spark
spark = init_spark("billing")
sc = spark.sparkContext

Ended spark session: 138.022882938 secs | default parallelism=2


In [ ]:
spark

In [ ]:
closing_day="20190414"

In [ ]:



# - - - - - - - - - -





In [ ]:
closing_day = "20190414"

#df_billing_p.show()


In [ ]:
# from pyspark.ml.feature import QuantileDiscretizer
# var = "billing_avg_days_bw_bills"
# var_out = "billing_avg_days_bw_bills_disc"


# discretizer = QuantileDiscretizer(numBuckets=5, inputCol=var, outputCol=var_out, relativeError=0)
# bucketizer = discretizer.fit(df_billing_p)
# df_billing_p = bucketizer.transform(df_billing_p)
# bucketizer.getSplits()

# df_billing_p = df_billing_p.withColumn("inc_billing_noinc_rgus_disc_2", when(col("billing_avg_days_bw_bills") <= 0, "<20")
#                                                                        .when(col("billing_avg_days_bw_bills") <=40, ">20 <40")
#                                                                        .when(col("billing_avg_days_bw_bills") >40,  ">40"))

# #df_billing_p.groupby("inc_billing_noinc_rgus_disc_2").agg(sql_count("*").alias("count")).show()

In [ ]:
# df_cust.columns : # ['num_cliente',  'nb_rgus_d',  'nb_rgus_d14',  'label',  'tgs_days_until_fecha_fin_dto',  'tgs_has_discount',  'nb_rgus_d28',  'nb_rgus_d42',  'diff_rgus_d_d14',  'diff_rgus_d_d28',  'diff_rgus_d_d42',
#  'flag_decremento_rgus',  'flag_incremento_rgus',  'flag_no_inc_rgus']

In [ ]:

def prepare_billing_cris(df, closing_day):

    NUM_BILLS = 5    

    remove_undefined_udf = udf(lambda milista: list([milista[ii] for ii in range(len(milista)) if milista[ii] == milista[ii] and milista[ii]!=None]), ArrayType(FloatType()))
    import numpy as np
    avg_days_bw_bills_udf = udf(lambda milista: float(np.mean([milista[ii + 1] - milista[ii] for ii in range(len(milista) - 1)])), FloatType())

    stddev_udf = udf(lambda milista: float(np.std(milista)), FloatType())
    #mean_udf = udf(lambda milista: float(np.mean(milista)), FloatType())


    mean_range_udf = udf(lambda lista,idx: float(np.mean(lista[:idx])), FloatType())
    negative_udf = udf(lambda lista: 1 if any([ii<0 for ii in lista]) else 0, IntegerType())

    # debt is negative when Vodafone has the debt
    df = (df.withColumn("aux_completed", array([_col for _col in df.columns if _col.endswith("days_since")]))
            .withColumn("aux_completed", remove_undefined_udf(col("aux_completed")))
            .withColumn("billing_nb_last_bills", when(col("aux_completed").isNotNull(), size(col("aux_completed"))).otherwise(0))
            .withColumn("billing_avg_days_bw_bills", when(col("billing_nb_last_bills") > 1, avg_days_bw_bills_udf(col("aux_completed"))).otherwise(-1.0))

           .withColumn("aux_invoice", array([_col for _col in df.columns if _col.endswith("_InvoiceCharges") and _col.startswith("Bill_N")]))
           .withColumn("aux_invoice", remove_undefined_udf(col("aux_invoice")))
           .withColumn("billing_std", when( ((col("aux_invoice").isNotNull()) & (col("billing_nb_last_bills") > 0)), stddev_udf("aux_invoice")).otherwise(-1.0))
           .withColumn("billing_mean", when((col("aux_invoice").isNotNull()) & (col("billing_nb_last_bills") > 0),
                                         mean_range_udf("aux_invoice","billing_nb_last_bills")).otherwise(-1.0))

          .withColumn("billing_current_debt", col('Bill_N1_InvoiceCharges') + col('Bill_N1_Debt_Amount') - col('Bill_N1_Amount_To_Pay'))

          .withColumn("aux_debt", array([_col for _col in df.columns if _col.endswith("_Debt_Amount") and _col.startswith("Bill_N")]))
          .withColumn("billing_any_vf_debt", negative_udf("aux_debt"))
          .withColumn("billing_current_vf_debt", when(col("billing_current_debt")<0,1).otherwise(0))
          .withColumn("billing_current_client_debt", when(col("billing_current_debt") > 0, 1).otherwise(0))
          .withColumn("billing_settle_debt", when( (col('Bill_N1_Debt_Amount') > col('billing_current_debt')) & (col("billing_current_vf_debt")==0),1) # deb is decreasing
                                            .otherwise(0))

          )


    return df


In [ ]:
from pyspark.sql.types import ArrayType, FloatType, StringType, BooleanType, IntegerType
from churn.utils.general_functions import amdocs_table_reader
df_billing = get_billing_df(spark, closing_day)
df_billing.show()

In [ ]:
df_billing_p = prepare_billing_cris(df_billing, closing_day)



In [ ]:
print("before", df_cust.count(), df_billing_p.count())
df_all = df_cust.join(df_billing_p, on=["num_cliente"], how="left")
df_all = df_all.cache()
print("after", df_all.count())

In [ ]:
from churn.analysis.triggers.billing.run_segment_billing import get_tariffs_dfs
from pykhaos.utils.date_functions import move_date_n_days


closing_day_c14 = move_date_n_cycles(closing_day, n=-14)
closing_day_c6 = move_date_n_cycles(closing_day, n=-6)

df_tarifas_c14 = get_tariffs_dfs(spark, closing_day_c14).drop("nb_superofertas").withColumnRenamed("tarifas_list", "tarifas_list_c14")
df_tarifas_c6 = get_tariffs_dfs(spark, closing_day_c6).drop("nb_superofertas").withColumnRenamed("tarifas_list", "tarifas_list_c6")

df_all2 = df_all.join(df_tarifas_c14, on=["num_cliente"], how="left")
df_all2 = df_all2.join(df_tarifas_c6, on=["num_cliente"], how="left")

In [ ]:
df_tarifas_c14.columns

In [ ]:
df_all2.select("num_cliente", "tarifas_list_c14", "tarifas_list_c6").show(50, truncate=False)

In [107]:
{k:v for k,v in spark.read.load("/user/csanc109/projects/triggers/trigger_billing_car/year=2019/month=4/day=14").dtypes if v not in ["bigint", "double", "int", "float"]}

{'Bill_N1_yearmonth_billing': 'string',
 'Bill_N2_yearmonth_billing': 'string',
 'Bill_N3_yearmonth_billing': 'string',
 'Bill_N4_yearmonth_billing': 'string',
 'Bill_N5_yearmonth_billing': 'string',
 'aux_completed': 'array<float>',
 'aux_debt': 'array<double>',
 'aux_invoice': 'array<float>',
 'diff_bw_bills': 'array<float>',
 'diff_bw_days_bills': 'array<float>',
 'equal_c6_c14': 'string',
 'nif_cliente': 'string',
 'num_cliente': 'string',
 'segment_nif': 'string',
 'tarifas_list': 'array<string>',
 'tarifas_list_c14': 'array<string>',
 'tarifas_list_c6': 'array<string>'}

In [ ]:
#Generated /data/attributes/vf_es/trigger_analysis/customer_master_unlabeled/year=2019/month=8/day=5 for closing_day 20190805
#revisar columna   'diff_rgus_map', rgus_diff_udf(col('rgus_list'), col('rgus_list_cycles_2')))
# TAREA para el 12 de agosto

In [69]:
df_tar_all = spark.read.load("/user/csanc109/projects/triggers/trigger_billing_car/year=2019/month=4/day=1414")
df_tar_count_numcliente = df_tar_all.select("nif_cliente").groupby("nif_cliente").agg(sql_count("*").alias("count"))

df_tar_all = df_tar_all.join(df_tar_count_numcliente, on=["nif_cliente"], how="right")
df_tar_all = df_tar_all.fillna(0, subset=["num_calls_w8"])
refprevalence_all = df_tar_all.select("label").rdd.map(lambda r: r["label"]).mean()
print("CHURN ALL {}%".format(refprevalence_all*100.0))

df_tar_all.select("label").groupby("label").agg(sql_count("*").alias("count")).show()

CHURN ALL 2.58905994041%
+-----+-------+
|label|  count|
+-----+-------+
|  0.0|3837240|
|  1.0| 101989|
+-----+-------+



In [ ]:
df_tar_all.groupby("num_calls_w8").agg(sql_count("*").alias("count")).show()

In [70]:
from churn.analysis.triggers.billing.run_segment_billing import filter_car
from pyspark.sql.functions import round, concat_ws

df_tar_f = filter_car(df_tar_all)
df_tar_f = df_tar_f.withColumn("num_ficticios", col('Bill_N1_num_ids_fict')+col('Bill_N2_num_ids_fict')+col('Bill_N3_num_ids_fict')+col('Bill_N4_num_ids_fict')+col('Bill_N5_num_ids_fict'))
df_tar_f = df_tar_f.withColumn("nb_tarifas", size(col('tarifas_list')))
df_tar_f = df_tar_f.withColumn("tgs_has_discount", when(col("tgs_has_discount")!=1,0).otherwise(col("tgs_has_discount")))
df_tar_f = df_tar_f.withColumn("tgs_has_discount_d84", when(col("tgs_has_discount_d84")!=1,0).otherwise(col("tgs_has_discount_d84")))


#df_tar_f.select("label").groupby("label").agg(sql_count("*").alias("count")).show()
# Filtering churn_cancellations_w8
# Filtering superoferta ON19
# +-----+-------+
# |label|  count|
# +-----+-------+
# |  0.0|3684672|
# |  1.0|  89072|
# +-----+-------+

Filtering churn_cancellations_w8
Filtering superoferta ON19


In [88]:
# COLS NUEVAS QUE GENERE Y QUE NO PROBE
# {'billing_max_days_bw_bills',
#  'billing_min_days_bw_bills',
#  'diff_bw_days_bills',
#  'flag_keep_rgus'}


df_tar_f = df_tar_f.withColumn("flag_keep_rgus",       when(((col("diff_rgus_d_d14") == 0) | (col("diff_rgus_d_d28") == 0) | (col("diff_rgus_d_d42") == 0) | (col("diff_rgus_d_d56") == 0) | (col("diff_rgus_d_d70") == 0) | (col("diff_rgus_d_d84") == 0) | (col("diff_rgus_d_d98") == 0)), 1).otherwise(0))
df_tar_f = df_tar_f.withColumn("mean_price_rgu",  col("Bill_N1_InvoiceCharges")/col('nb_rgus_d'))


df_tar_rule = (df_tar_f#.where(col("billing_nb_last_bills")==5)
#                     .where(col("billing_any_vf_debt")==1)
#                     .where(col("billing_current_vf_debt")==0)
#                     .where(col('nb_tarifas') == col('num_tarifas_c14'))
#                      .where( (col("greatest_diff_bw_bills")> (0.3*col("billing_mean"))) & (col("greatest_diff_bw_bills") > 0.3 * col("Bill_N5_InvoiceCharges")))
#                       .where(col('num_calls_w8')>0)
               .where(  ((col("flag_keep_rgus")==1) & (col("tgs_has_discount")==0)) | (col("flag_decremento_rgus")==1))
                 #.where(col("greatest_diff_bw_bills")>(0.3*col("billing_mean")))
                     .where(col('num_calls_w8')>2)
                     #.where( col('billing_std') > (0.3*col("billing_mean")))
                     .where(col("greatest_diff_bw_bills") > 0.10 * col("Bill_N5_InvoiceCharges"))
               
               
                    # .where(col('BILLING_POSTPAID_w8')>0)
                     #.where(col('BILLING_POSTPAID_w8')>0)
#                      .where(col("flag_keep_rgus")==1)
#                       .where(col('nb_rgus_d')>1)
#                     .where(col("equal_c6_c14")==0)
#                     .where(col("tgs_has_discount_d84")==col("tgs_has_discount"))
                   # .where( col('billing_std') > (0.25*col("billing_mean")))
#                    .where( ((col('billing_std') > (0.4*col("Bill_N2_InvoiceCharges"))) | (col('billing_std') > (0.4*col("Bill_N1_InvoiceCharges")))))
#                     .where( (col('Bill_N1_InvoiceCharges') > (1.25*col("Bill_N2_InvoiceCharges"))) | (col('Bill_N1_InvoiceCharges') > 1.15*col("billing_mean")))
#                      .where( col('billing_mean') > 30)
#                      .where(col('Bill_N1_days_since')<35)
#                    .where(col("num_ficticios")>0)
#                     .where(  
                             # (col("Bill_N1_InvoiceCharges") > (1.2 * col("Bill_N4_InvoiceCharges"))) |
#                               (col("Bill_N2_InvoiceCharges") > (1.2 * col("Bill_N3_InvoiceCharges"))) |
#                               (col("Bill_N1_InvoiceCharges") > (1.2 * col("Bill_N3_InvoiceCharges"))) 
#                              )    
#                     .where(  ((col("Bill_N2_InvoiceCharges") > (1.2 * col("Bill_N4_InvoiceCharges"))) |
#                               (col("Bill_N1_InvoiceCharges") > (1.2 * col("Bill_N4_InvoiceCharges")))) |
                             
#                              ((col("Bill_N2_InvoiceCharges") > (1.2 * col("Bill_N5_InvoiceCharges"))) |  
#                               (col("Bill_N1_InvoiceCharges") > (1.2 * col("Bill_N5_InvoiceCharges")))) |
                                                        
#                              ((col("Bill_N2_InvoiceCharges") > (1.2 * col("Bill_N3_InvoiceCharges"))) |  
#                               (col("Bill_N1_InvoiceCharges") > (1.2 * col("Bill_N3_InvoiceCharges")))) 
#                              )    
         )

refprevalence = df_tar_rule.select("label").rdd.map(lambda r: r["label"]).mean()
print("LIFT = {} || CHURN {}% || VOLUME={}".format(refprevalence/refprevalence_all, refprevalence*100.0, df_tar_rule.count()))

from pyspark.sql.functions import mean as sql_mean, stddev as sql_std
#df_tar_rule.select("label").groupby("label").agg(sql_count("*").alias("count")).show()

#df_tar_rule.groupby("BILLING_POSTPAID_w8").agg(sql_count("*").alias("count")).show()

LIFT = 2.13852558412 || CHURN 5.53677092139% || VOLUME=18928


In [86]:
df_tar_rule.select("label", "Bill_N1_InvoiceCharges_disc").groupby("label", "Bill_N1_InvoiceCharges_disc").agg(sql_count("*").alias("count")).show()

AnalysisException: u"cannot resolve '`Bill_N1_InvoiceCharges_disc`' given input columns: [nb_rgus_d70, Bill_N1_Tax_Amount, Bill_N4_net_charges, billing_avg_days_bw_bills, diff_bw_bills, day, BUCKET_w8, num_cliente, Bill_N5_yearmonth_billing, nb_rgus_d56, Bill_N1_num_ids_fict, Bill_N5_net_charges, nb_rgus_d42, Bill_N1_num_facturas, Bill_N1_Debt_Amount, billing_any_vf_debt, Bill_N5_Tax_Amount, tgs_days_until_fecha_fin_dto, Bill_N5_days_since, count, Bill_N4_yearmonth_billing, num_calls_w8, Bill_N5_Amount_To_Pay, BILLING_POSTPAID_w8, tarifas_list, MI_VODAFONE_w8, Bill_N4_days_since, billing_nb_last_bills, diff_rgus_d_d14, Bill_N2_InvoiceCharges, label, flag_incremento_rgus, billing_std, Bill_N3_InvoiceCharges, Bill_N5_InvoiceCharges, Bill_N3_Debt_Amount, Bill_N5_num_facturas, Bill_N2_yearmonth_billing, least_diff_bw_bills, Bill_N3_yearmonth_billing, OTHER_CUSTOMER_INFOSATIS_START_AVERIA_w8, DSL_FIBER_INCIDENCES_AND_SUPPORT_w8, Bill_N4_Tax_Amount, OTHER_CUSTOMER_INFORMATION_MANAGEMENT_w8, 

In [91]:
## from pyspark.ml.feature import QuantileDiscretizer

for var in ["Bill_N1_InvoiceCharges", "mean_price_rgu"]:
    if var+"_disc" in df_tar_rule.columns:
        df_tar_rule = df_tar_rule.drop(var+"_disc")
    discretizer = QuantileDiscretizer(numBuckets=4, inputCol=var, outputCol=var+"_disc", relativeError=0.001)
    bucketizer = discretizer.fit(df_tar_rule)
    df_tar_rule = bucketizer.transform(df_tar_rule)
    print(bucketizer.getSplits())

# [-inf, 29.25, 49.72, 75.19, inf]
df_tar_rule.select("label", "mean_price_rgu_disc").groupby("label", "mean_price_rgu_disc").agg(sql_count("*").alias("count"), ).show()


[-inf, 53.33, 81.3, 121.8, inf]
[-inf, 15.575999999999999, 23.05, 36.505, inf]
+-----+-------------------+-----+
|label|mean_price_rgu_disc|count|
+-----+-------------------+-----+
|  0.0|               null|    1|
|  1.0|                1.0|  262|
|  0.0|                1.0| 4471|
|  1.0|                0.0|  201|
|  1.0|                2.0|  311|
|  0.0|                0.0| 4535|
|  1.0|                3.0|  274|
|  0.0|                2.0| 4432|
|  0.0|                3.0| 4441|
+-----+-------------------+-----+



In [93]:
df_tar_rule.select("label", "mean_price_rgu_disc").groupby("label", "mean_price_rgu_disc").agg(sql_count("*").alias("count")).sort(desc("count")).show()


+-----+-------------------+-----+
|label|mean_price_rgu_disc|count|
+-----+-------------------+-----+
|  0.0|                0.0| 4535|
|  0.0|                1.0| 4471|
|  0.0|                3.0| 4441|
|  0.0|                2.0| 4432|
|  1.0|                2.0|  311|
|  1.0|                3.0|  274|
|  1.0|                1.0|  262|
|  1.0|                0.0|  201|
|  0.0|               null|    1|
+-----+-------------------+-----+



In [8]:

from pyspark.sql.functions import trim

df_tar_f = df_tar_f.withColumn("cohort", lit(""))

# - - - - - - - 
# REGLA 1
# - - - - - - - 
df_tar_f = df_tar_f.withColumn("cohort1", when( ((col("billing_nb_last_bills")==5) 
                                              & (col("equal_c6_c14")==0) 
                                              & (col("tgs_has_discount_d84")>col("tgs_has_discount")) 
                                              & (col('billing_std') > (0.4*col("billing_mean")))
                                              & (col('billing_mean') > 60)
                                              & (col('Bill_N1_days_since')<75)
                                              & (col("Bill_N1_InvoiceCharges") > col("Bill_N5_InvoiceCharges"))), "A").otherwise(None))
# - - - - - - - 
# REGLA 2
# - - - - - - - 
df_tar_f = df_tar_f.withColumn("cohort2", when( (col("billing_nb_last_bills")==5) &
                   (col("greatest_diff_bw_bills")>50) &
                   (col("equal_c6_c14")==0) &
                   (col("tgs_has_discount_d84")==col("tgs_has_discount")) &
                   ( col('billing_std') > (0.5*col("billing_mean"))) &
                   ( col('billing_mean') > 35) & 
                   (col('Bill_N1_days_since')<35) &
                   (  ((col("Bill_N2_InvoiceCharges") > (1.35 * col("Bill_N4_InvoiceCharges"))) & 
                             (col("Bill_N1_InvoiceCharges") > (1.35 * col("Bill_N4_InvoiceCharges")))) |
                             ((col("Bill_N2_InvoiceCharges") > (1.35 * col("Bill_N5_InvoiceCharges"))) &  
                             (col("Bill_N1_InvoiceCharges") > (1.35 * col("Bill_N5_InvoiceCharges")))) 
                            ), "B").otherwise(None))  

# - - - - - - - 
# REGLA 3
# - - - - - - - 
df_tar_f = df_tar_f.withColumn("cohort3", when( (col("billing_nb_last_bills")==5) &
                    (col("greatest_diff_bw_bills")>50) &
                    (col('num_calls_w8')>1) &
                    (col('nb_rgus_d')<=col('nb_rgus_d84')) &
                    (col("tgs_has_discount_d84")==col("tgs_has_discount")) &
                    (  ((col("Bill_N2_InvoiceCharges") > (2.5 * col("Bill_N4_InvoiceCharges"))) |
                              (col("Bill_N1_InvoiceCharges") > (2.5 * col("Bill_N4_InvoiceCharges")))) &
                             ((col("Bill_N2_InvoiceCharges") > (2.5 * col("Bill_N5_InvoiceCharges"))) |  
                              (col("Bill_N1_InvoiceCharges") > (2.5 * col("Bill_N5_InvoiceCharges")))) &               
                             ((col("Bill_N2_InvoiceCharges") > (2.5 * col("Bill_N3_InvoiceCharges"))) |  
                              (col("Bill_N1_InvoiceCharges") > (2.5 * col("Bill_N3_InvoiceCharges")))) 
                             ),"C"   ).otherwise(None))


# - - - - - - - 
# REGLA 4
# - - - - - - - 
df_tar_f = df_tar_f.withColumn("cohort4",when( (col("greatest_diff_bw_bills")>40) &
                     (col('num_calls_w8')>3) &
                     ( col('billing_std') > (0.3*col("billing_mean"))) &
                     (col('BILLING_POSTPAID_w8')>0), "D").otherwise(None))


df_tar_f = df_tar_f.withColumn("cohort", lit(""))
df_tar_f = df_tar_f.withColumn("cohort", trim(concat_ws(" ", col("cohort1"), col("cohort2"), col("cohort3"), col("cohort4"))))
df_tar_f = df_tar_f.withColumn("in_cohort", when( (col("cohort1").isNotNull() | col("cohort2").isNotNull() | col("cohort3").isNotNull() | col("cohort4").isNotNull()), 1).otherwise(0))

# df_tar_f.select("label", "cohort1").groupby("label", "cohort1").agg(sql_count("*").alias("count")).show()
# df_tar_f.select("label", "cohort2").groupby("label", "cohort2").agg(sql_count("*").alias("count")).show()
# df_tar_f.select("label", "cohort").groupby("label", "cohort").agg(sql_count("*").alias("count")).show()
df_tar_f.select("label", "cohort1", "cohort2", "cohort3", "cohort4").groupby("label", "cohort1", "cohort2", "cohort3", "cohort4").agg(sql_count("*").alias("count")).show()

df_tar_f.select("label", "in_cohort").groupby("label", "in_cohort").agg(sql_count("*").alias("count")).show()

refprevalence_incohort = df_tar_f.where(col("in_cohort")==1).select("label").rdd.map(lambda r: r["label"]).mean()

print("LIFT = {} || CHURN {}% || VOLUME={}".format(refprevalence_incohort/refprevalence_all, refprevalence_incohort*100.0, df_tar_f.where(col("in_cohort")==1).count()))


+-----+-------+-------+-------+-------+-------+
|label|cohort1|cohort2|cohort3|cohort4|  count|
+-----+-------+-------+-------+-------+-------+
|  0.0|   null|   null|   null|   null|3670342|
|  0.0|   null|   null|      C|   null|   1598|
|  1.0|   null|   null|      C|      D|     58|
|  1.0|   null|      B|      C|   null|      7|
|  1.0|   null|   null|      C|   null|    107|
|  1.0|   null|      B|   null|      D|     14|
|  1.0|   null|   null|   null|   null|  88164|
|  0.0|   null|      B|   null|   null|   2286|
|  1.0|   null|      B|      C|      D|     11|
|  0.0|      A|   null|   null|      D|     35|
|  0.0|   null|   null|   null|      D|   8896|
|  1.0|      A|   null|   null|   null|     36|
|  1.0|   null|      B|   null|   null|    147|
|  0.0|   null|      B|      C|      D|     86|
|  1.0|   null|   null|   null|      D|    521|
|  0.0|   null|      B|   null|      D|    191|
|  0.0|   null|      B|      C|   null|    130|
|  0.0|      A|   null|   null|   null| 

In [ ]:
df_tar_f.where(col("num_ficticios")>0).select("num_cliente", "label", "greatest_diff_bw_bills",
               round("billing_mean",2).alias("mean"), 
               round('Bill_N1_InvoiceCharges',2).alias("B1"),
               round('Bill_N2_InvoiceCharges',2).alias("B2"), 
               round('Bill_N3_InvoiceCharges',2).alias("B3"),  
               round('Bill_N4_InvoiceCharges',2).alias("B4"),  
               round('Bill_N5_InvoiceCharges',2).alias("B5"), 
               round('billing_std',2).alias("std"), 
              'billing_avg_days_bw_bills',  
              col('tgs_days_until_fecha_fin_dto').alias("days_nodcto"), 
              col('Bill_N1_days_since').alias("days_B1"),
              'tgs_has_discount', 
              'tgs_has_discount_d28',
              'tgs_has_discount_d56', 
              'tgs_has_discount_d84', 
              col('billing_nb_last_bills').alias("nb")).sort(asc("label")).show(150,truncate=False)

In [ ]:
df_tar.select("num_cliente", "label", "greatest_diff_bw_bills",
               round("billing_mean",2).alias("mean"), 
               round('Bill_N1_InvoiceCharges',2).alias("B1"),
               round('Bill_N2_InvoiceCharges',2).alias("B2"), 
               round('Bill_N3_InvoiceCharges',2).alias("B3"),  
               round('Bill_N4_InvoiceCharges',2).alias("B4"),  
               round('Bill_N5_InvoiceCharges',2).alias("B5"), 
               round('billing_std',2).alias("std"), 
              'billing_avg_days_bw_bills',  
              col('tgs_days_until_fecha_fin_dto').alias("days_nodcto"), 
              col('Bill_N1_days_since').alias("days_B1"),
              'tgs_has_discount', 
              'tgs_has_discount_d28',
              'tgs_has_discount_d56', 
              'tgs_has_discount_d84', 
              col('billing_nb_last_bills').alias("nb")).sort(asc("label")).show(150,truncate=False)

In [ ]:
from churn.delivery.checkers_class import checker, check_join_car, check_extra_feats, check_scores_new, \
    check_levers_model, check_car, check_car_preparado, check_churn_reasons_table, check_delivery_file, check_delivery_table,\
check_model_evaluation, check_model_output_entry
    
from churn.delivery.delivery_constants import SECTION_CHURN_DELIV_ENTRY
from churn.delivery.delivery_constants import MODEL_NAME_DELIV_CHURN
from churn.delivery.printers_class import format_table_msg, format_table_msg
from functools import partial

closing_day = "20190721"
training_closing_date = "20190531to20190531"
 
checker_mod_out_delivery_entry_obj = checker(spark, partial(check_model_output_entry, spark, MODEL_NAME_DELIV_CHURN, closing_day, training_closing_date),
                                            partial(format_table_msg), closing_day, SECTION_CHURN_DELIV_ENTRY)


checker_mod_out_delivery_entry_obj.check_func()
checker_mod_out_delivery_entry_obj.print_msg()

In [94]:
predict_closing_day = "20190809"
from pykhaos.utils.date_functions import get_next_dow
return_feed_execution = get_next_dow(weekday=3, from_date=dt.datetime.strptime(predict_closing_day, "%Y%m%d")).strftime("%Y%m%d")
return_feed_execution

'20190814'

In [112]:
dt.datetime.today().strftime("%Y%m%d")

'20190814'

In [110]:

from pykhaos.utils.date_functions import move_date_n_days, get_previous_cycle, move_date_n_cycles
import datetime as dt

today_start  = "20190501"
today_end    = "20191231"

today = today_start

while today <= today_end:
    
    closing_day = get_previous_cycle(move_date_n_days(today, n=-6))
    
    training_day = get_previous_cycle(move_date_n_cycles(move_date_n_days(today, n=-4), n=-8))
    
    print("today={} {} | closing_day={} training_day={}".format(today, dt.datetime.strptime(today, "%Y%m%d").strftime('%A')[:3], closing_day, training_day))
    
    today = move_date_n_days(today, n=+1)
    







today=20190501 Wed | closing_day=20190421 training_day=20190221
today=20190502 Thu | closing_day=20190421 training_day=20190221
today=20190503 Fri | closing_day=20190421 training_day=20190221
today=20190504 Sat | closing_day=20190421 training_day=20190221
today=20190505 Sun | closing_day=20190421 training_day=20190228
today=20190506 Mon | closing_day=20190421 training_day=20190228
today=20190507 Tue | closing_day=20190430 training_day=20190228
today=20190508 Wed | closing_day=20190430 training_day=20190228
today=20190509 Thu | closing_day=20190430 training_day=20190228
today=20190510 Fri | closing_day=20190430 training_day=20190228
today=20190511 Sat | closing_day=20190430 training_day=20190228
today=20190512 Sun | closing_day=20190430 training_day=20190307
today=20190513 Mon | closing_day=20190430 training_day=20190307
today=20190514 Tue | closing_day=20190507 training_day=20190307
today=20190515 Wed | closing_day=20190507 training_day=20190307
today=20190516 Thu | closing_day=2019050

In [23]:
#spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_fbb").show()
#df_param.show()

closing_day = "20190721"
segment = "fbb"
sql_query = "select msisdn, scoring, model_executed_at from parquet.`/data/attributes/vf_es/model_outputs/model_scores/model_name={}` where predict_closing_date = '{}'".format(
    "churn_preds_" + segment, closing_day)

df = spark.sql(sql_query)
df.show()

+------+-------+-----------------+
|msisdn|scoring|model_executed_at|
+------+-------+-----------------+
+------+-------+-----------------+



In [22]:
spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_fbb").select("executed_at").distinct().show()

+-------------------+
|        executed_at|
+-------------------+
|2019-07-14 23:53:55|
|2019-08-16 00:12:44|
|2019-08-15 04:07:18|
|2019-07-16 02:58:40|
|2019-07-13 06:07:03|
|2019-07-18 13:35:50|
|2019-08-16 07:55:09|
|2019-07-14 20:25:45|
|2019-07-13 17:19:21|
|2019-08-17 06:20:41|
|2019-07-13 23:42:01|
|2019-07-15 04:38:53|
|2019-07-15 10:28:49|
|2019-07-13 11:34:52|
|2019-07-15 19:19:54|
|2019-07-14 01:54:42|
|2019-08-16 16:55:11|
|2019-08-15 15:43:41|
|2019-07-12 22:26:57|
|2019-07-10 18:59:54|
+-------------------+
only showing top 20 rows



In [17]:
spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_mobileandfbb").where(col("training_closing_date")=="20190521to20190521").distinct().collect()
#df_param.show()

[Row(executed_at=u'2019-08-02 09:50:16', model_level=u'service', training_closing_date=u'20190521to20190521', target=u'port', model_path=u'/data/udf/vf_es/churn/models/prediction_tr20190521to20190521_tt20190721_horizon8_on20190802_095016', metrics_path=u'-', metrics_train=u'roc=0.7885914284895749;avg_score=0.3745269255996354;sd_score=0.29529045947533117;skewness_score=0.44355841166974;kurtosis_score=1.9291663901729443;min_score=0.003324185381314986;max_score=0.9693865718371922', metrics_test=u'roc=0.7539426044726851;avg_score=0.3627625441573877;sd_score=0.22159400592473041;skewness_score=0.3238206294544013;kurtosis_score=2.523807255662095;min_score=0.003324185381314986;max_score=0.9693865718371922', varimp=u'tgs_days_until_f_fin_bi;tgs_days_until_fecha_fin_dto;tgs_days_since_f_inicio_bi;total_max_dias_hasta_penal_srv_pending_end_date;dias_desde_movil_fx_first;dias_desde_fixed_fx_first;total_penal_srv_pending_n1_penal_amount;Bill_N1_Tax_Amount;dias_desde_fx_tariff;Bill_N1_InvoiceCharges

In [13]:

# pred name: prediction_fbb_tr20190521to20190521_tt20190721_horizon8_on20190812_101649
closing_day = "20190807"
segment = "fbb"
horizon = 8



sql_query = "select msisdn, scoring, model_executed_at from parquet.`/data/attributes/vf_es/model_outputs/model_scores/model_name={}` where predict_closing_date = '{}'".format(
    "churn_preds_" + segment, closing_day)

df = spark.sql(sql_query)

exe_list = df.select("model_executed_at").distinct().rdd.map(lambda xx: xx[0]).collect()

sql_query = "select * from parquet.`/data/attributes/vf_es/model_outputs/model_parameters/model_name={}` where executed_at in ('{}') sort by training_closing_date desc, executed_at desc".format(
    "churn_preds_" + segment,
    "','".join(exe_list),
)

df_param = spark.sql(sql_query)

df_param = df_param.withColumn("horizon", regexp_extract(col("extra_info"), "horizon=([0-9]*)(;|$)", 1))

if df_param.where(col("horizon").isNotNull()).take(1):
    existing_models = df_param.where(col("horizon") == horizon)
else:
    print("horizon value not found as expected")
    existing_models = df_param  # do not filter by horizon

existing_models = existing_models.select("training_closing_date", "executed_at").rdd.collect()

models = []
for mm in existing_models:
    models.append(mm.asDict())


print("models")
models

#from churn.delivery.model_outputs_formatter import build_pred_name, insert_to_model_outputs
# build_pred_name

#build_pred_name(spark,  "20190721", segment="fbb", horizon=8)



models


[{'executed_at': u'2019-08-15 04:07:18', 'training_closing_date': u'20190531'}]

In [15]:
spark.read.load("/data/attributes/vf_es/trigger_analysis/customer_master/year=2019/month=6/day=7").groupby("segment_nif").agg(sql_count("*").alias("count")).show()

+--------------+-------+
|   segment_nif|  count|
+--------------+-------+
|Standalone_FBB| 157149|
|         Other| 625216|
|    Convergent|2131634|
|   Mobile_only| 956273|
+--------------+-------+



In [7]:
closing_day = "20190414"

partition_date = "year={}/month={}/day={}".format(int(closing_day[:4]), int(closing_day[4:6]), int(closing_day[6:]))


tr_ccc_all = spark.read.parquet('/data/attributes/vf_es/trigger_analysis/ccc/' + partition_date)

bill_cols = [c for c in tr_ccc_all.columns if 'billing' in c.lower()]

bill_cols

['BILLING_POSTPAID_w2',
 'BILLING_POSTPAID_w4',
 'BILLING_POSTPAID_w8',
 'inc_BILLING_POSTPAID_w2vsw2',
 'inc_BILLING_POSTPAID_w4vsw4']

In [9]:
from pyspark.sql.functions import mean as sql_avg, greatest

closing_day = "20190521"

partition_date = "year={}/month={}/day={}".format(int(closing_day[:4]), int(closing_day[4:6]), int(closing_day[6:]))

tr_base = (
    spark.read.parquet('/data/attributes/vf_es/trigger_analysis/customer_master/' + partition_date).filter(col('segment_nif') != 'Pure_prepaid').withColumn('blindaje', lit('none')).withColumn(
        'blindaje', when((col('tgs_days_until_fecha_fin_dto') >= 0) & (col('tgs_days_until_fecha_fin_dto') <= 60), 'soft').otherwise(col('blindaje'))).withColumn('blindaje', when(
        (col('tgs_days_until_fecha_fin_dto') > 60), 'hard').otherwise(col('blindaje'))).select('nif_cliente', 'blindaje', 'nb_rgus', 'nb_tv_services_nif', 'label'))

print("KK", tr_base.select(sql_avg('label').alias('churn_rate')).rdd.first()['churn_rate'], tr_base.count())
tr_base = tr_base.drop_duplicates(["nif_cliente"])

tr_churn_ref = tr_base.select(sql_avg('label').alias('churn_rate')).rdd.first()['churn_rate']


('KK', 0.03245234192118541, 3899811)


In [63]:
df_tt_all = spark.read.parquet('/user/csanc109/projects/triggers/trigger_billing_car/'+partition_date)
[col_ for col_ in df_tt_all.columns if "segment" in col_]

['segment_nif']

In [74]:
from pyspark.sql.functions import mean as sql_avg, greatest

closing_day = "20190521"

# !!! HAS DUPLICATES "/user/csanc109/projects/triggers/trigger_exploration/year=2019/month=5/day=21/time=110803/"
# model1 is Cristina's model
df_model1 = (spark.read.load("/user/csanc109/projects/triggers/trigger_exploration/year=2019/month=5/day=21/time=200859/")
                            .select(['NIF_CLIENTE', 'calib_model_score', 'label']).withColumnRenamed("calib_model_score", "calib_model_score_model1")                                                                         
                 .withColumnRenamed("label", "label_model1")).where(col("NIF_CLIENTE").isNotNull()).drop_duplicates(["NIF_CLIENTE"])


# model2 is Alvaro's model
df_model2 = (spark.read.load("/data/udf/vf_es/churn/triggers/scores50K_20190521_rf").select(['NIF_CLIENTE', 'calib_model_score', "label"])
                                                                         .withColumnRenamed("calib_model_score", "calib_model_score_model2")
                                                                         .withColumnRenamed("label", "label_model2")).where(col("NIF_CLIENTE").isNotNull()).drop_duplicates(["NIF_CLIENTE"])

          
print("STATS") # just a check, count must be 50k in both cases
print('\tNIFs model2 = {}'.format(df_model2.count()))
print('\tNIFs model1 = {}'.format(df_model1.count()))

# compute common nifs
df_inner = df_model1.join(df_alv, on=["nif_cliente"], how="inner")
common_nifs = df_inner.count()
print("\t Common nifs = {}".format(common_nifs))

# NIFs in Cris' model noiit included in Alvaro's model
df_model1_no_model2 = df_model2.select("nif_cliente").join(df_model1.select("nif_cliente"), ['nif_cliente'], 'right').where(df_model2['nif_cliente'].isNull())

# NIFs in Alvaro's model not included in Cris' model
df_model2_no_model1 = df_model1.select("nif_cliente").join(df_alv.select("nif_cliente"), ['nif_cliente'], 'right').where(df_model1['nif_cliente'].isNull())

# Absurd print, since new NIFs are 50k - common, but good for checking
print("NIFs nuevos model1 sobre model2 = {} [check={}]".format(df_model1_no_model2.count(), df_model1_no_model2.count() + common_nifs))
print("NIFs nuevos model2 sobre model1 = {} [check={}]".format(df_model2_no_model1.count(), df_model2_no_model1.count() + common_nifs))

from churn.analysis.triggers.trigger_billing_ccc.trigger_smart_exploration import get_customer_master

_, tr_volume_ref, tr_churn_ref = get_customer_master(spark, closing_day)


print("Reference base: churn_rate={:.2f} volume={}".format(100.0*tt_churn_ref, tr_volume_ref))
                                                                                                               
   

STATS
	NIFs model2 = 50000
	NIFs model1 = 50000
	 Common nifs = 15699
NIFs nuevos model1 sobre model2 = 34301 [check=50000]
NIFs nuevos model2 sobre model1 = 34301 [check=50000]
Reference base: churn_rate=3.24 volume=3941466


In [78]:

pos_max_udf = udf(
        lambda milist: sorted([(vv, idx) for idx, vv in enumerate(milist)], key=lambda tup: tup[0], reverse=True)[0][1],
        IntegerType())

df_all = df_model2.join(df_model1, on=["nif_cliente"], how="full").fillna(0)  

df_all = df_all.withColumn("calib_model_score", greatest(*["calib_model_score_model1", "calib_model_score_model2"]))

df_all = df_all.withColumn("score_array", array(["calib_model_score_model1", "calib_model_score_model2"]))  
df_all = df_all.withColumn("pos_max", pos_max_udf(col("score_array")))  


df_all = df_all.withColumn("label", greatest(*["label_model1", "label_model2"])).where(col("NIF_CLIENTE").isNotNull()).drop_duplicates(["NIF_CLIENTE"])
df_all = df_all.withColumn("model", when(col("pos_max")==0, "csanc109").when(col("pos_max")==1, "asaezco").otherwise("unknown"))

df_all = df_all.drop("pos_max", "score_array")
df_all = df_all.sort(desc("calib_model_score"))

df_all = df_all.cache()

num_rows = df_all.count()

print("After cache {}".format(num_rows))


After cache 84301


In [79]:
#df_all.groupby("model").agg(sql_count("*").alias("count")).show()
df_all.columns

['NIF_CLIENTE',
 'calib_model_score_model2',
 'label_model2',
 'calib_model_score_model1',
 'label_model1',
 'calib_model_score',
 'label',
 'model']

In [87]:
df_kk = spark.read.load("/user/csanc109/projects/triggers/trigger_exploration_combined/year=2019/month=5/day=21")
df_kk.count()

84301

In [86]:
path_to_save = "/user/csanc109/projects/triggers/trigger_exploration_combined/"
closing_day = "20190521"
df_all = df_all.withColumn("day", lit(int(closing_day[6:])))
df_all = df_all.withColumn("month", lit(int(closing_day[4:6])))
df_all = df_all.withColumn("year", lit(int(closing_day[:4])))
start_time = time.time()
print("Started saving - {} for closing_day={}".format(path_to_save, closing_day))
(df_all.coalesce(1).write.partitionBy('year', 'month', 'day').mode("append").format("parquet").save(path_to_save))
print("Elapsed time saving {} minutes".format((time.time()-start_time)/60.0))

Started saving - /user/csanc109/projects/triggers/trigger_exploration_combined/ for closing_day=20190521
Elapsed time saving 0.00830854972204 minutes


In [68]:

myschema = df_all.schema

for ii in [50000, 40000, 30000, 20000, 15000, 12000, 10000, 5000, 3000, 2000, 1000, 2500, 8000, 16000, 25000, 35000, 45000, num_rows]:
    
    start_time_ii = time.time()
    df_top = spark.createDataFrame(df_all.head(ii),  schema=myschema)
    churn_rate_top =  df_top.select(sql_avg('label').alias('churn_rate')).rdd.first()['churn_rate']
    print("TOP{} - LIFT={:.2f} churn_rate={:.2f}% churn_rate_ref={:.2f}%".format(ii, 
                                                                                          churn_rate_top/tt_churn_ref,
                                                                                          churn_rate_top*100.0,
                                                                                          tt_churn_ref*100.0, 
                                                                                          (time.time()-start_time_ii)/60.0))
    inner_nifs = df_top.join(df_inner, on=["NIF_CLIENTE"], how="inner").select("NIF_CLIENTE").count()
    from_model1_nifs = df_top.join(df_cris_no_alv, on=["NIF_CLIENTE"], how="inner").select("NIF_CLIENTE").count()
    from_model2_nifs = df_top.join(df_alv_no_cris, on=["NIF_CLIENTE"], how="inner").select("NIF_CLIENTE").count()
    
    print("\t COMMON={} [{:.2f}%] || FROM_MODEL1={} [{:.2f}%] || FROM_MODEL2={} [{:.2f}%]".format(inner_nifs, 100.0*inner_nifs/ii, from_model1_nifs, 100.0*from_model1_nifs/ii, from_model2_nifs, 100.0*from_model2_nifs/ii))
                                                           
                                                           


TOP50000 - LIFT=7.23 churn_rate=23.41% churn_rate_ref=3.24%
	 COMMON=13538 [27.08%] || FROM_MODEL1=14566 [29.13%] || FROM_MODEL2=23773 [47.55%]
TOP40000 - LIFT=8.02 churn_rate=25.98% churn_rate_ref=3.24%
	 COMMON=11748 [29.37%] || FROM_MODEL1=11565 [28.91%] || FROM_MODEL2=18027 [45.07%]
TOP30000 - LIFT=9.16 churn_rate=29.67% churn_rate_ref=3.24%
	 COMMON=9329 [31.10%] || FROM_MODEL1=8669 [28.90%] || FROM_MODEL2=12736 [42.45%]
TOP20000 - LIFT=10.96 churn_rate=35.52% churn_rate_ref=3.24%
	 COMMON=6534 [32.67%] || FROM_MODEL1=5581 [27.91%] || FROM_MODEL2=8176 [40.88%]
TOP15000 - LIFT=12.35 churn_rate=40.00% churn_rate_ref=3.24%
	 COMMON=5101 [34.01%] || FROM_MODEL1=4060 [27.07%] || FROM_MODEL2=5969 [39.79%]
TOP12000 - LIFT=13.46 churn_rate=43.61% churn_rate_ref=3.24%
	 COMMON=4273 [35.61%] || FROM_MODEL1=3061 [25.51%] || FROM_MODEL2=4667 [38.89%]
TOP10000 - LIFT=14.35 churn_rate=46.51% churn_rate_ref=3.24%
	 COMMON=3715 [37.15%] || FROM_MODEL1=2391 [23.91%] || FROM_MODEL2=3822 [38.22%]
TO

In [72]:
myschema = df_model1.schema

df_model1 = (spark.read.load("/user/csanc109/projects/triggers/trigger_exploration/year=2019/month=5/day=21/time=200859/")
                            .select(['NIF_CLIENTE', 'model_score', 'label']))
myschema = df_model1.schema

df_model1 = df_model1.sort(desc("model_score"))


for ii in [50000, 40000, 30000, 20000, 16000, 12000, 10000, 5000, 3000, 2000, 1000, 2500, 8000, 15000, 25000, 35000, 45000, num_rows]:
    
    start_time_ii = time.time()
    df_top = spark.createDataFrame(df_model1.head(ii),  schema=myschema)
    churn_rate_top =  df_top.select(sql_avg('label').alias('churn_rate')).rdd.first()['churn_rate']
    print("TOP{} - LIFT={:.2f} churn_rate={:.2f}% churn_rate_ref={:.2f}%".format(ii, 
                                                                                          churn_rate_top/tt_churn_ref,
                                                                                          churn_rate_top*100.0,
                                                                                          tt_churn_ref*100.0, 
                                                                                          (time.time()-start_time_ii)/60.0))
                

TOP50000 - LIFT=6.45 churn_rate=20.91% churn_rate_ref=3.24%
TOP40000 - LIFT=6.92 churn_rate=22.42% churn_rate_ref=3.24%
TOP30000 - LIFT=7.73 churn_rate=25.06% churn_rate_ref=3.24%
TOP20000 - LIFT=9.06 churn_rate=29.36% churn_rate_ref=3.24%
TOP16000 - LIFT=9.64 churn_rate=31.23% churn_rate_ref=3.24%
TOP12000 - LIFT=10.79 churn_rate=34.96% churn_rate_ref=3.24%
TOP10000 - LIFT=11.56 churn_rate=37.44% churn_rate_ref=3.24%
TOP5000 - LIFT=14.78 churn_rate=47.88% churn_rate_ref=3.24%
TOP3000 - LIFT=16.97 churn_rate=54.97% churn_rate_ref=3.24%
TOP2000 - LIFT=18.10 churn_rate=58.65% churn_rate_ref=3.24%
TOP1000 - LIFT=19.20 churn_rate=62.20% churn_rate_ref=3.24%
TOP2500 - LIFT=17.54 churn_rate=56.84% churn_rate_ref=3.24%
TOP8000 - LIFT=12.67 churn_rate=41.05% churn_rate_ref=3.24%
TOP15000 - LIFT=9.85 churn_rate=31.90% churn_rate_ref=3.24%
TOP25000 - LIFT=8.36 churn_rate=27.08% churn_rate_ref=3.24%
TOP35000 - LIFT=7.27 churn_rate=23.56% churn_rate_ref=3.24%
TOP45000 - LIFT=6.67 churn_rate=21.60%

In [ ]:
#incremental

df_C = df_cris.select("nif_cliente").join(df_cris2.select("nif_cliente"), ['nif_cliente'], 'right').where(df_cris['nif_cliente'].isNull())
print("RESUMEN - INCREMENTALES", df_C.count())

In [88]:
arget_base = spark.read.parquet('/user/hive/warehouse/tests_es.db/jvmm_amdocs_ids_' + '20190821')

AnalysisException: u'Unable to infer schema for Parquet. It must be specified manually.;'